# Processor for Feedback from Tabbycat

In the administrator area of tabbycat, go to setup -> Export XML. Replace the filename 'Tournament.xml' in the next cell with your exported file.

Below, you will have to modify the questions to export to feedback.

In [ ]:
import xml.dom.minidom
Load_XML = xml.dom.minidom.parse('Iron23.xml')

In [ ]:
attr = Load_XML.getElementsByTagName('adjudicator')[0].attributes['name']

In [ ]:
from xml.dom.minicompat import NodeList
from xml.dom.minidom import Element
import itertools


def adj_to_feedback(adjs: NodeList[Element]) -> list[dict]:
    filled_feedback = [
            {
                'to': adj.attributes['name'].nodeValue,
                'score': fb.attributes['score'].nodeValue,
                'question': answer.attributes['question'].nodeValue,
                'answer': answer.childNodes[0].nodeValue,
                'source': (fb.attributes['source-adjudicator'] if 'source-adjudicator' in fb.attributes else fb.attributes['source-team']).nodeValue,
                'debate': fb.attributes['debate'].nodeValue
            }
            for adj in adjs for fb in adj.childNodes for answer in fb.childNodes 
         ]
    # make sure to include scores w/o text for final aggregation
    filled_set = { (fb['to'], fb['question'], fb['source'], fb['debate']) for fb in filled_feedback }
    for adj in adjs:
        to = adj.attributes['name'].nodeValue
        for fb in adj.childNodes:
            score = fb.attributes['score'].nodeValue
            source = (fb.attributes['source-adjudicator'] if 'source-adjudicator' in fb.attributes else fb.attributes['source-team']).nodeValue
            debate = fb.attributes['debate'].nodeValue
            for q in Load_XML.getElementsByTagName('question'):
                q_id = q.attributes['id'].nodeValue
                if (to, q_id, source, debate) not in filled_set:
                    filled_feedback.append({
                        'to': to,
                        'score': score,
                        'question': q_id,
                        'answer': '',
                        'source': source,
                        'debate': debate
                    })
    return filled_feedback

In [ ]:
import pandas as pd

In [ ]:
df_feedback = pd.DataFrame(adj_to_feedback(Load_XML.getElementsByTagName('adjudicator')))

In [ ]:
[(q.attributes['id'].nodeValue, q.attributes['name'].nodeValue, q.nodeValue) for q in Load_XML.getElementsByTagName('question')]

In [ ]:
# Note just above this cell which question (e.g. 'Q4') is the open feedback question you want to export and modify
# the next line accordingly.
df_feedback[df_feedback.question == 'Q6'].to_csv('feedback.csv')